In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=e4ba7af490f2f24cf7354c67bc32a15803fdabbcac67752a59d1ffda8e9eb75c
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Facebook Live sellers in Thailand") \
    .getOrCreate()

In [ ]:
from re import MULTILINE
file_path ='Live_20210128.csv'
spark_df = spark.read.option('header',True).option('inferSchema',True).csv(file_path)
#
# .option("quote", "\"")\
# .option("escape", "\"")\

In [ ]:
spark_df.show(vertical=True)

-RECORD 0---------------------------
 status_id        | 1               
 status_type      | video           
 status_published | 4/22/2018 6:00  
 num_reactions    | 529             
 num_comments     | 512             
 num_shares       | 262             
 num_likes        | 432             
 num_loves        | 92              
 num_wows         | 3               
 num_hahas        | 1               
 num_sads         | 1               
 num_angrys       | 0               
 Column1          | NULL            
 Column2          | NULL            
 Column3          | NULL            
 Column4          | NULL            
-RECORD 1---------------------------
 status_id        | 2               
 status_type      | photo           
 status_published | 4/21/2018 22:45 
 num_reactions    | 150             
 num_comments     | 0               
 num_shares       | 0               
 num_likes        | 150             
 num_loves        | 0               
 num_wows         | 0               
 

In [ ]:
spark_df.count()

7050

In [ ]:
spark_df.printSchema()

root
 |-- status_id: integer (nullable = true)
 |-- status_type: string (nullable = true)
 |-- status_published: string (nullable = true)
 |-- num_reactions: integer (nullable = true)
 |-- num_comments: integer (nullable = true)
 |-- num_shares: integer (nullable = true)
 |-- num_likes: integer (nullable = true)
 |-- num_loves: integer (nullable = true)
 |-- num_wows: integer (nullable = true)
 |-- num_hahas: integer (nullable = true)
 |-- num_sads: integer (nullable = true)
 |-- num_angrys: integer (nullable = true)
 |-- Column1: string (nullable = true)
 |-- Column2: string (nullable = true)
 |-- Column3: string (nullable = true)
 |-- Column4: string (nullable = true)



In [ ]:
from pyspark.sql.types import (IntegerType, DoubleType, StringType,FloatType,DateType)
from pyspark.sql.functions import col,udf,lit,expr,collect_set,monotonically_increasing_id,row_number
import pyspark.sql.functions as func
from pyspark.sql.window import Window
from pyspark.sql.functions import (avg, col, countDistinct, split, isnan, when, count, udf, round as sqlround)
import math
import numpy as np


In [ ]:
# checking null data
from pyspark.sql.functions import col, count, isnan, lit, sum
import random
def count_not_null(c, nan_as_null=False):
    """Use conversion between boolean and integer
    - False -> 0
    - True ->  1
    """
    pred = col(c).isNotNull() & (~isnan(c) if nan_as_null else lit(True))
    return sum(pred.cast("integer")).alias(c)

spark_df.agg(*[count_not_null(c) for c in spark_df.columns]).show()

+---------+-----------+----------------+-------------+------------+----------+---------+---------+--------+---------+--------+----------+-------+-------+-------+-------+
|status_id|status_type|status_published|num_reactions|num_comments|num_shares|num_likes|num_loves|num_wows|num_hahas|num_sads|num_angrys|Column1|Column2|Column3|Column4|
+---------+-----------+----------------+-------------+------------+----------+---------+---------+--------+---------+--------+----------+-------+-------+-------+-------+
|     7050|       7050|            7050|         7050|        7050|      7050|     7050|     7050|    7050|     7050|    7050|      7050|      0|      0|      0|      0|
+---------+-----------+----------------+-------------+------------+----------+---------+---------+--------+---------+--------+----------+-------+-------+-------+-------+



In [ ]:
spark_df.columns

['status_id',
 'status_type',
 'status_published',
 'num_reactions',
 'num_comments',
 'num_shares',
 'num_likes',
 'num_loves',
 'num_wows',
 'num_hahas',
 'num_sads',
 'num_angrys',
 'Column1',
 'Column2',
 'Column3',
 'Column4']

In [ ]:
# Xóa các cột dữ liệu dư thừa
drop_cols=[]
spark_df = spark_df.drop('Column1','Column2','Column3','Column4')
spark_df.show()

+---------+-----------+----------------+-------------+------------+----------+---------+---------+--------+---------+--------+----------+
|status_id|status_type|status_published|num_reactions|num_comments|num_shares|num_likes|num_loves|num_wows|num_hahas|num_sads|num_angrys|
+---------+-----------+----------------+-------------+------------+----------+---------+---------+--------+---------+--------+----------+
|        1|      video|  4/22/2018 6:00|          529|         512|       262|      432|       92|       3|        1|       1|         0|
|        2|      photo| 4/21/2018 22:45|          150|           0|         0|      150|        0|       0|        0|       0|         0|
|        3|      video|  4/21/2018 6:17|          227|         236|        57|      204|       21|       1|        1|       0|         0|
|        4|      photo|  4/21/2018 2:29|          111|           0|         0|      111|        0|       0|        0|       0|         0|
|        5|      photo|  4/18/2018

In [ ]:
len(spark_df.columns)

12

In [ ]:
spark_df.select('status_type').distinct().show()

+-----------+
|status_type|
+-----------+
|       link|
|     status|
|      video|
|      photo|
+-----------+



In [ ]:
def add_column(df,lst,name):
  lst_udf = func.udf( \
    lambda indx: lst[indx-1])
  df1 = df.withColumn( \
    "num_id", row_number().over( \
    Window.orderBy(monotonically_increasing_id())))

  # Create a new column by calling the user defined function
  df1 = df1.withColumn(name, \
                        lst_udf('num_id'))
  df1= df1.drop('num_id')
  return df1

In [ ]:
list_values = spark_df.select("status_published").rdd.flatMap(lambda x: x).collect()

In [ ]:
print(list_values)

['4/22/2018 6:00', '4/21/2018 22:45', '4/21/2018 6:17', '4/21/2018 2:29', '4/18/2018 3:22', '4/18/2018 2:14', '4/18/2018 0:24', '4/17/2018 7:42', '4/17/2018 3:33', '4/11/2018 4:53', '4/10/2018 1:01', '4/9/2018 2:06', '4/8/2018 5:10', '4/8/2018 2:23', '4/5/2018 9:23', '4/1/2018 5:16', '3/30/2018 8:28', '3/26/2018 8:28', '3/23/2018 7:09', '3/22/2018 1:25', '3/21/2018 8:40', '3/21/2018 7:46', '3/20/2018 1:54', '3/20/2018 1:28', '3/20/2018 0:15', '3/19/2018 22:34', '3/17/2018 8:07', '3/17/2018 7:47', '3/12/2018 5:51', '3/9/2018 7:45', '3/9/2018 5:55', '3/7/2018 13:10', '3/7/2018 6:13', '3/7/2018 4:46', '3/5/2018 8:32', '3/5/2018 2:48', '3/2/2018 9:55', '2/28/2018 8:19', '2/28/2018 1:43', '2/26/2018 8:50', '2/25/2018 10:13', '2/21/2018 0:19', '2/19/2018 0:11', '2/18/2018 7:15', '2/17/2018 22:30', '2/17/2018 4:15', '2/15/2018 3:47', '2/14/2018 8:23', '2/13/2018 1:16', '2/12/2018 3:28', '2/11/2018 5:55', '2/10/2018 7:54', '2/9/2018 1:48', '2/8/2018 9:14', '2/7/2018 8:35', '2/6/2018 8:46', '2/

In [ ]:
spark_df.columns

['status_id',
 'status_type',
 'status_published',
 'num_reactions',
 'num_comments',
 'num_shares',
 'num_likes',
 'num_loves',
 'num_wows',
 'num_hahas',
 'num_sads',
 'num_angrys']

In [ ]:

date = []
time = []
for i in range(len(list_values)):
  val = list_values[i].split(' ')
  date.append(val[0])
  time.append(val[1])
print(date)
print(time)
print(type(time))

['4/22/2018', '4/21/2018', '4/21/2018', '4/21/2018', '4/18/2018', '4/18/2018', '4/18/2018', '4/17/2018', '4/17/2018', '4/11/2018', '4/10/2018', '4/9/2018', '4/8/2018', '4/8/2018', '4/5/2018', '4/1/2018', '3/30/2018', '3/26/2018', '3/23/2018', '3/22/2018', '3/21/2018', '3/21/2018', '3/20/2018', '3/20/2018', '3/20/2018', '3/19/2018', '3/17/2018', '3/17/2018', '3/12/2018', '3/9/2018', '3/9/2018', '3/7/2018', '3/7/2018', '3/7/2018', '3/5/2018', '3/5/2018', '3/2/2018', '2/28/2018', '2/28/2018', '2/26/2018', '2/25/2018', '2/21/2018', '2/19/2018', '2/18/2018', '2/17/2018', '2/17/2018', '2/15/2018', '2/14/2018', '2/13/2018', '2/12/2018', '2/11/2018', '2/10/2018', '2/9/2018', '2/8/2018', '2/7/2018', '2/6/2018', '2/5/2018', '2/4/2018', '2/3/2018', '2/2/2018', '2/2/2018', '2/2/2018', '1/28/2018', '1/28/2018', '1/26/2018', '1/26/2018', '1/26/2018', '1/25/2018', '1/23/2018', '1/22/2018', '1/22/2018', '1/21/2018', '1/20/2018', '1/20/2018', '1/19/2018', '1/19/2018', '1/18/2018', '1/18/2018', '1/17/20

In [ ]:
# from pyspark.sql.functions import to_date

# date2 = []
# for i in range(len(date)):
#   val = date[i].split('/')
#   date_val = val[2]+"-"
#   if len(val[0])==1 :
#     date_val += "0"+ val[0]+"-"
#   else :
#     date_val += val[0] + "-"

#   if len(val[1])==1 :
#     date_val += "0"+ val[1]
#   else :
#     date_val += val[1]
#   date2.append(date_val)
# print(date2)

In [ ]:

# #tokenizer text
# from pyspark.sql.functions import row_number, monotonically_increasing_id
# from pyspark.sql.window import Window
# from pyspark.sql import functions as func
# from pyspark.sql.types import StringType,FloatType,DateType

# times_udf = func.udf( \
#   lambda indx: time[indx-1],StringType())
# df1 = spark_df.withColumn( \
#   "num_id", row_number().over( \
#   Window.orderBy(monotonically_increasing_id())))

# # Create a new column by calling the user defined function
# spark_df = df1.withColumn('time', \
#                        times_udf('num_id'))
# spark_df.drop('num_id').show()

In [ ]:

# dates_udf = func.udf( \
#   lambda indx: date2[indx-1])
# df1 = spark_df.withColumn( \
#   "num_id", row_number().over( \
#   Window.orderBy(monotonically_increasing_id())))

# # Create a new column by calling the user defined function
# spark_df = df1.withColumn('Date', \
#                        dates_udf('num_id'))
# spark_df.drop('num_id').show()

In [ ]:
# spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

# spark_df = spark_df.withColumn('new_date', to_date(spark_df.Date, 'yyyy-mm-dd').cast(DateType()))
# spark_df.show()

In [ ]:
spark_df = spark_df.drop('status_id','num_id','status_published')

In [ ]:
# spark_df= spark_df.replace(['link','status','video','photo'],['1','2','3','4']\
#                 ,'status_type')
# spark_df.select("status_type").distinct().show()

In [ ]:
# from pyspark.sql.types import IntegerType
# spark_df = spark_df.withColumn("status_type", spark_df["status_type"].cast(IntegerType()))

In [ ]:
spark_df = add_column(spark_df,time,'time')

In [ ]:
time_values = spark_df.select("time").rdd.flatMap(lambda x: x).collect()

In [ ]:
time = []
for i in range(len(time_values)):
  val = time_values[i].split(':')
  time.append((val[0]))
print(time)
print(type(time))

['6', '22', '6', '2', '3', '2', '0', '7', '3', '4', '1', '2', '5', '2', '9', '5', '8', '8', '7', '1', '8', '7', '1', '1', '0', '22', '8', '7', '5', '7', '5', '13', '6', '4', '8', '2', '9', '8', '1', '8', '10', '0', '0', '7', '22', '4', '3', '8', '1', '3', '5', '7', '1', '9', '8', '8', '9', '3', '8', '9', '7', '6', '7', '5', '13', '13', '8', '8', '9', '9', '9', '9', '23', '9', '9', '9', '8', '5', '9', '22', '22', '0', '9', '9', '8', '23', '6', '10', '8', '23', '2', '6', '1', '8', '1', '22', '1', '0', '22', '1', '0', '0', '21', '23', '12', '11', '22', '22', '3', '1', '3', '3', '0', '23', '3', '6', '3', '2', '9', '2', '5', '2', '1', '22', '6', '5', '22', '2', '10', '3', '21', '22', '8', '23', '3', '1', '23', '0', '22', '22', '1', '1', '3', '0', '8', '8', '1', '0', '23', '21', '9', '0', '7', '6', '0', '3', '22', '6', '22', '4', '2', '23', '7', '6', '3', '2', '5', '8', '8', '9', '4', '7', '2', '2', '6', '10', '10', '1', '0', '0', '4', '23', '11', '11', '8', '9', '9', '7', '2', '20', '0', '2

In [ ]:
spark_df = add_column(spark_df,time,'Hour')
spark_df = spark_df.drop('time')

In [ ]:
# times_udf = func.udf( \
#   lambda indx: time[indx-1])
# df1 = spark_df.withColumn( \
#   "num_id", row_number().over( \
#   Window.orderBy(monotonically_increasing_id())))

# # Create a new column by calling the user defined function
# spark_df = df1.withColumn('Hour', \
#                        times_udf('num_id'))
# spark_df= spark_df.drop('num_id')
# spark_df.show()


In [ ]:
from pyspark.sql.types import (IntegerType, DoubleType, StringType)
spark_df = spark_df.withColumn('Hour', spark_df.Hour.cast(IntegerType()))
spark_df.show()

+-----------+-------------+------------+----------+---------+---------+--------+---------+--------+----------+----+
|status_type|num_reactions|num_comments|num_shares|num_likes|num_loves|num_wows|num_hahas|num_sads|num_angrys|Hour|
+-----------+-------------+------------+----------+---------+---------+--------+---------+--------+----------+----+
|      video|          529|         512|       262|      432|       92|       3|        1|       1|         0|   6|
|      photo|          150|           0|         0|      150|        0|       0|        0|       0|         0|  22|
|      video|          227|         236|        57|      204|       21|       1|        1|       0|         0|   6|
|      photo|          111|           0|         0|      111|        0|       0|        0|       0|         0|   2|
|      photo|          213|           0|         0|      204|        9|       0|        0|       0|         0|   3|
|      photo|          217|           6|         0|      211|        5| 

In [ ]:
spark_df.printSchema()

root
 |-- status_type: string (nullable = true)
 |-- num_reactions: integer (nullable = true)
 |-- num_comments: integer (nullable = true)
 |-- num_shares: integer (nullable = true)
 |-- num_likes: integer (nullable = true)
 |-- num_loves: integer (nullable = true)
 |-- num_wows: integer (nullable = true)
 |-- num_hahas: integer (nullable = true)
 |-- num_sads: integer (nullable = true)
 |-- num_angrys: integer (nullable = true)
 |-- Hour: integer (nullable = true)



In [ ]:
def label_encode(df, input_col, output_col):
    distinct_values = df.select(input_col).distinct().collect()
    mapping = {row[input_col]: index for index, row in enumerate(distinct_values)}
    mapping_broadcast = spark.sparkContext.broadcast(mapping)
    def label_encode_udf(value):
        return mapping_broadcast.value.get(value, None)
    spark.udf.register("label_encode_udf", label_encode_udf, FloatType())
    df = df.withColumn(output_col, lit(None).cast(FloatType()))
    for value, index in mapping.items():
        condition = col(input_col) == lit(value)
        df = df.withColumn(output_col, when(condition, lit(index).cast(FloatType())).otherwise(col(output_col)))
    df = df.drop(input_col).withColumnRenamed(output_col, input_col)
    return df

In [ ]:
df = spark_df
df = label_encode(df,'status_type','type')
df.printSchema()

root
 |-- num_reactions: integer (nullable = true)
 |-- num_comments: integer (nullable = true)
 |-- num_shares: integer (nullable = true)
 |-- num_likes: integer (nullable = true)
 |-- num_loves: integer (nullable = true)
 |-- num_wows: integer (nullable = true)
 |-- num_hahas: integer (nullable = true)
 |-- num_sads: integer (nullable = true)
 |-- num_angrys: integer (nullable = true)
 |-- Hour: integer (nullable = true)
 |-- status_type: float (nullable = true)



In [ ]:
df.select("status_type").distinct().show()

+-----------+
|status_type|
+-----------+
|        2.0|
|        3.0|
|        1.0|
|        0.0|
+-----------+



In [ ]:
# hàm nhân giữa giá trị hàng i với hàng index, trả về tuple
def math_cal(lines,index,i):
    return (index, float(lines[i]) * float(lines[index]))

In [ ]:
def linear_regression(data_train, X_train, y_train):

  #khởi tạo 2 list chứa dữ liệu
    value_X = [] #phụ thuộc
    value_y = [] #độc lập

  #Tạo ra View tạm thời để lưu trữ dữ liệu
    data_train.createOrReplaceTempView("data")
    columns_data =""

  #Đọc tên cột trong list x_train và y_train
    for i in range(len(X_train)):
        columns_data = columns_data + X_train[i] + ","
    columns_data = columns_data + y_train[0]

  #Sử dụng sql để query dữ liệu từ bảng data
    sql_data_train = spark.sql("SELECT "+ columns_data +" FROM data")

  #chuyển bộ dữ liệu sang dạng rdd
    data_train_rdd = sql_data_train.rdd

  #Đếm số lượng dòng dữ liệu train và thêm vào value_x
    n = data_train_rdd.count()
    value_X.append(float(n))

  #map dữ liệu từng dòng thành kiểu tuple,
  #thêm index và chuyển toàn bộ kiểu dữ liệu của dòng sang  float
    map_X_sum = data_train_rdd.map(lambda lines: \
     [(str(index),float(lines[index])) for index in range(len(lines))])

  #làm phẳng dữ liệu thành pair-key sau đó
  #reduce bằng cách cộng giá trị và lưu giá trị lại data_X_sum
    flatmap_X_sum = map_X_sum.flatMap(lambda lines: lines)
    reduce_X_sum = flatmap_X_sum.reduceByKey(lambda a,b: a+b)
    data_X_sum=reduce_X_sum.collect()

  #thêm vào list giá trị value_X và values_y
    for i in range(len(data_X_sum)):
      #lọc giá trị cuối để thêm vào value_y
        if(i == len(data_X_sum)-1):
            value_y.append(data_X_sum[i][1])
        else:
            value_X.append(data_X_sum[i][1])

    for i in range (len(data_X_sum)-1):
      #sử dụng hàm math_cal để nhân hàng thứ i với các hàng của dữ liệu
        map_X_vol = data_train_rdd.map(lambda lines:\
         [math_cal(lines, index, i) for index in range(len(lines))])

      #xử lý dữ liệu như đã thực hiện với map_X_sum
        flatmap_X_vol = map_X_vol.flatMap(lambda lines: lines)
        reduce_X_vol = flatmap_X_vol.reduceByKey(lambda a,b: a+b)
        data_X_vol = reduce_X_vol.collect()

      #Thêm giá trị sum của hàng thứ i vào value_X
        value_X.append(data_X_sum[i][1])

      #Thêm giá trị các cột ở hàng thứ i vào value_X
        for j in range(len(data_X_vol)):
            if(j == len(data_X_vol) - 1): #lọc giá trị cuối
                value_y.append(data_X_vol[j][1])
            else:
                value_X.append(data_X_vol[j][1])

    #tạo vector_X và sử dụng np chuyển value_X thành dạng array kiểu dữ liệu float
    #sau đó reshape với độ dài +1
    vector_X = np.asarray(value_X, dtype=np.float32)
    vector_X_convert=np.reshape(vector_X, (len(X_train)+1, len(X_train)+1))

    #tạo vector_X và tương tự như vector_X
    #reshape thành dạng (độ dài + 1, 1)
    vector_y = np.asarray(value_y, dtype=np.float32)
    vector_y_convert=np.reshape(vector_y, (len(X_train)+1, 1))

    #sử dụng hàm lstsq của numpy để tính coefficient
    coefficient = np.linalg.lstsq(vector_X_convert, vector_y_convert)[0]

    return coefficient


In [ ]:
def predict(data_test, coefficient):
    y=coefficient[0].item()
    #nhân dữ liệu của data_test và coefficient để cho ra dự đoán
    for i in range(len(data_test.columns)-1):
        y = y + data_test[i+1]*coefficient[i+1].item()
    return y/len(data_test.columns)

In [ ]:
def R_square(df_predict,actual,predict):
    avg = df_predict.select(actual).summary('mean').collect()[0][1]

    df_predict_avg= df_predict.withColumn("avg_log",\
     (col(actual)-avg)*(col(actual)-avg))

    avg_log=df_predict_avg.select('avg_log').\
    summary('mean').collect()[0][1]

    sst=float(avg_log)*float(df_predict_avg.\
                                      select('avg_log').count())


    df_predict_avg= df_predict.withColumn\
     ("predict_avg_log",(col(actual)-col(predict))*\
      (col(actual)-col(predict)))

    avg_predict_avg_log = df_predict_avg.select(\
    'predict_avg_log').summary('mean').collect()[0][1]

    sse = float(avg_predict_avg_log)*\
    float(df_predict_avg.select('predict_avg_log').count())

    R_square=1-(sse/sst)

    return R_square

In [ ]:
def RMSD(df_predict, n,actual,predict):

    df_predict_avg= df_predict.withColumn("predict_avg_log",(col(actual)-col(predict))\
                                          *(col(actual)-col(predict)))
    avg_predict_avg_log = df_predict_avg.select('predict_avg_log').summary('mean').collect()[0][1]
    sse = float(avg_predict_avg_log)*float(df_predict_avg.select('predict_avg_log').count())

    return math.sqrt(sse/n)

In [ ]:
df.show()

+-------------+------------+----------+---------+---------+--------+---------+--------+----------+----+-----------+
|num_reactions|num_comments|num_shares|num_likes|num_loves|num_wows|num_hahas|num_sads|num_angrys|Hour|status_type|
+-------------+------------+----------+---------+---------+--------+---------+--------+----------+----+-----------+
|          529|         512|       262|      432|       92|       3|        1|       1|         0|   6|        2.0|
|          150|           0|         0|      150|        0|       0|        0|       0|         0|  22|        3.0|
|          227|         236|        57|      204|       21|       1|        1|       0|         0|   6|        2.0|
|          111|           0|         0|      111|        0|       0|        0|       0|         0|   2|        3.0|
|          213|           0|         0|      204|        9|       0|        0|       0|         0|   3|        3.0|
|          217|           6|         0|      211|        5|       1|    

In [ ]:
X_train = df.columns
X_train.remove('status_type')

y_train = ['status_type']

In [ ]:

data_train = df.limit(int(df.count() * 0.8))
data_test = df.subtract(data_train)
coefficient = linear_regression(\
                                data_train\
                                ,X_train\
                                ,y_train)

<ipython-input-36-8e950ea18576>:76: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coefficient = np.linalg.lstsq(vector_X_convert, vector_y_convert)[0]


In [ ]:
df_predict = data_test.withColumn("type_predict",predict(data_test,coefficient))
df_predict.show(30)

+-------------+------------+----------+---------+---------+--------+---------+--------+----------+----+-----------+--------------------+
|num_reactions|num_comments|num_shares|num_likes|num_loves|num_wows|num_hahas|num_sads|num_angrys|Hour|status_type|        type_predict|
+-------------+------------+----------+---------+---------+--------+---------+--------+----------+----+-----------+--------------------+
|           16|           1|         0|       16|        0|       0|        0|       0|         0|  23|        3.0|  0.3292896557832137|
|          285|        1293|       295|      167|      111|       1|        5|       0|         1|   7|        2.0| -3.5598978000889474|
|           69|           8|         0|       60|        0|       2|        0|       1|         6|  23|        3.0| 0.33664135121084243|
|           32|           0|         1|       32|        0|       0|        0|       0|         0|   2|        2.0| 0.24687949515414817|
|          101|        1753|       227|  

In [ ]:
predict_values = df_predict.select('type_predict').rdd.flatMap(lambda x: x).collect()

In [ ]:
print(predict_values)

[0.3292896557832137, -3.5598978000889474, 0.33664135121084243, 0.24687949515414817, -5.355764556615626, 0.2917849690777736, -6.682502207525514, -0.6470542909529203, 0.24673726368796037, 0.3251064387470912, -0.030344084825297945, -5.981857861115052, 0.2606333957790577, 0.26004694229050074, 0.20467243742181876, 0.2432687791618016, -0.5894192476685434, 0.26076402300069196, -4.462386153495159, -3.1786917533308126, 0.2339378948914104, 0.1007251294054599, 0.3235326853596648, 0.22945914827207822, -2.263429509201325, 0.2923523620265769, 0.27262945560373325, -6.241988150856328, -10.048163540510854, -3.2319216967582842, -0.9100907089033915, 0.25630873295647855, -3.4222225618081707, 0.2704616134830179, -0.8842281863554408, 0.17207094890181907, -23.76106314069991, 0.25636841817785433, 0.26624883372013836, 0.20467088082395177, 0.24777831713700751, 0.24658986683987844, -13.4342416347486, -0.8744373717560269, 0.2670411398046946, 0.24084484608002557, 0.27070892638981936, -2.5934734572087605, 0.2584464

In [ ]:
val_count = df.select("status_type").distinct().count()
alter_predict = []
for i in range(len(predict_values)):
  val = abs(float(round(predict_values[i] % val_count)))
  alter_predict.append(val)
print(alter_predict)


[0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 1.0, 3.0, 0.0, 0.0, 4.0, 2.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 4.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 2.0, 2.0, 1.0, 3.0, 0.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 3.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 3.0, 3.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 3.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 3.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 3.0, 3.0, 4.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 3.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 3.0, 3.0, 0.0, 0.0, 0.0, 1.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 3.0, 3.0, 0.0, 3.0, 0.0, 1.0, 3.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 0.0, 3.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 4.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 4.0, 2.0, 1.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 3.0, 2.0, 4.0, 0.0, 3.0, 0.0, 1.0, 0.0, 0.0,

In [ ]:
df_predict = add_column(df_predict,alter_predict,'type_predict')
df_predict.show()

+-------------+------------+----------+---------+---------+--------+---------+--------+----------+----+-----------+------------+
|num_reactions|num_comments|num_shares|num_likes|num_loves|num_wows|num_hahas|num_sads|num_angrys|Hour|status_type|type_predict|
+-------------+------------+----------+---------+---------+--------+---------+--------+----------+----+-----------+------------+
|           16|           1|         0|       16|        0|       0|        0|       0|         0|  23|        3.0|         0.0|
|          285|        1293|       295|      167|      111|       1|        5|       0|         1|   7|        2.0|         0.0|
|           69|           8|         0|       60|        0|       2|        0|       1|         6|  23|        3.0|         0.0|
|           32|           0|         1|       32|        0|       0|        0|       0|         0|   2|        2.0|         0.0|
|          101|        1753|       227|       83|       15|       1|        2|       0|         0

In [ ]:
df_predict.select('status_type','type_predict').show()

+-----------+------------+
|status_type|type_predict|
+-----------+------------+
|        3.0|         0.0|
|        2.0|         0.0|
|        3.0|         0.0|
|        2.0|         0.0|
|        2.0|         3.0|
|        1.0|         0.0|
|        2.0|         1.0|
|        3.0|         3.0|
|        3.0|         0.0|
|        3.0|         0.0|
|        3.0|         4.0|
|        2.0|         2.0|
|        3.0|         0.0|
|        2.0|         0.0|
|        3.0|         0.0|
|        3.0|         0.0|
|        1.0|         3.0|
|        1.0|         0.0|
|        2.0|         4.0|
|        2.0|         1.0|
+-----------+------------+
only showing top 20 rows



In [ ]:
print("R_square: "+ str(R_square(df_predict,'status_type','type_predict')))
print("RMSD: "+str(RMSD(df_predict,data_test.count(),'status_type','type_predict')))

R_square: -8.914472608339938
RMSD: 2.235874370159066


In [ ]:
def cal_accuracy(df,actual, predict):
  actual_values = df.select(actual).rdd.flatMap(lambda x: x).collect()
  predict_values = df.select(predict).rdd.flatMap(lambda x: x).collect()
  val = 0
  for i in range(len(actual_values)):
    if str(actual_values[i]) == str(predict_values[i]):
      val= val + 1
  return val/len(actual_values)*100

In [ ]:
print(cal_accuracy(df_predict,'status_type','type_predict'))

9.87012987012987


Dự đoán số lượng chia sẻ

In [ ]:
df2 = df.drop('status_type')

In [ ]:
X_train2 = df2.columns
X_train2.remove('num_shares')

y_train2 = ['num_shares']

In [ ]:

data_train2 = df2.limit(int(df2.count() * 0.8))
data_test2 = df2.subtract(data_train2)
coefficient2 = linear_regression(\
                                data_train2\
                                ,X_train2\
                                ,y_train2)

<ipython-input-36-8e950ea18576>:76: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coefficient = np.linalg.lstsq(vector_X_convert, vector_y_convert)[0]


In [ ]:
df_predict2 = data_test2.withColumn("share_predict",func.abs(predict(data_test2,coefficient2)))
df_predict2.show(30)

+-------------+------------+----------+---------+---------+--------+---------+--------+----------+----+------------------+
|num_reactions|num_comments|num_shares|num_likes|num_loves|num_wows|num_hahas|num_sads|num_angrys|Hour|     share_predict|
+-------------+------------+----------+---------+---------+--------+---------+--------+----------+----+------------------+
|          386|        1912|       394|      246|      137|       1|        2|       0|         0|   1|2615.2035444110634|
|          255|         505|       187|      168|       80|       5|        1|       0|         1|   7|441.27029234021904|
|         1730|         161|         4|     1710|       12|       8|        0|       0|         0|  11|2654.1054740071295|
|          237|          67|         1|      233|        3|       1|        0|       0|         0|   6|287.08626198619606|
|           43|          17|         0|       41|        1|       1|        0|       0|         0|   0| 43.43433899879456|
|           10| 

In [ ]:
predict_values2 = df_predict2.select('share_predict').rdd.flatMap(lambda x: x).collect()

In [ ]:
print(predict_values2)

[2615.2035444110634, 441.27029234021904, 2654.1054740071295, 287.08626198619606, 43.43433899879456, 16.62843852043152, 186.6803717494011, 197.6233213663101, 2374.441802893579, 1314.8114379897713, 73.71215504556895, 59.179409408569335, 1570.0644797518848, 76.05078711509705, 1238.9618406578898, 891.6651758521796, 4188.46933260262, 76.44308295696973, 1859.3788097858428, 53.15868022590875, 132.00806263685226, 74.71774190664291, 219.0942250162363, 6.830949699878692, 24.964406931400298, 2438.5136601552367, 4216.161077387631, 37.00511537790298, 35.668342062830924, 1796.9217223867774, 13.714327985048294, 139.85447510927915, 142.27760982513428, 827.4139150828123, 175.54130077511073, 468.1579143702984, 4159.969975787401, 1085.4217564702035, 8.956678187847137, 185.65689168423415, 208.10037925690412, 576.1374298229814, 221.36105416566133, 37.16892536878586, 50.6548167526722, 27.19000670313835, 491.7391451388597, 47.420859958231446, 63.84886917322874, 73.20245064347982, 2232.991161867976, 55.614636

In [ ]:
df_predict2.select('num_shares','share_predict').show()

+----------+------------------+
|num_shares|     share_predict|
+----------+------------------+
|       394|2615.2035444110634|
|       187|441.27029234021904|
|         4|2654.1054740071295|
|         1|287.08626198619606|
|         0| 43.43433899879456|
|         0| 16.62843852043152|
|         0| 186.6803717494011|
|         4| 197.6233213663101|
|       451| 2374.441802893579|
|        11|1314.8114379897713|
|         3| 73.71215504556895|
|         0|59.179409408569335|
|       157|1570.0644797518848|
|         0| 76.05078711509705|
|        83|1238.9618406578898|
|       198| 891.6651758521796|
|       154|  4188.46933260262|
|         1| 76.44308295696973|
|       348|1859.3788097858428|
|        15| 53.15868022590875|
+----------+------------------+
only showing top 20 rows



In [ ]:
print("R_square: "+ str(R_square(df_predict2,'num_shares','share_predict')))
print("RMSD: "+str(RMSD(df_predict2,data_test2.count(),'num_shares','share_predict')))

R_square: -259.4480456925176
RMSD: 2573.9603885644287


In [ ]:
def avg_error_rate2(df,actual, predict):
  actual_values = df.select(actual).rdd.flatMap(lambda x: x).collect()
  predict_values = df.select(predict).rdd.flatMap(lambda x: x).collect()
  val = 0
  for i in range(len(actual_values)):
    if float(actual_values[i]) == 0:
      val += float(predict_values[i])
    else:
      val +=abs(float(actual_values[i])-float(predict_values[i]))/float(actual_values[i])
  return val/len(actual_values)

In [ ]:
print(avg_error_rate2(df_predict2,'num_shares','share_predict'))

63.21226081297641


Dự đoán số lượng bình luận

In [ ]:
X_train3 = df2.columns
X_train3.remove('num_comments')

y_train3 = ['num_comments']

In [ ]:

data_train3 = df2.limit(int(df2.count() * 0.8))
data_test3 = df2.subtract(data_train3)
coefficient3 = linear_regression(\
                                data_train3\
                                ,X_train3\
                                ,y_train3)

<ipython-input-36-8e950ea18576>:76: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coefficient = np.linalg.lstsq(vector_X_convert, vector_y_convert)[0]


In [ ]:
df_predict3 = data_test3.withColumn("comments_predict",func.abs(predict(data_test3,coefficient3)))
df_predict3.show(30)

+-------------+------------+----------+---------+---------+--------+---------+--------+----------+----+------------------+
|num_reactions|num_comments|num_shares|num_likes|num_loves|num_wows|num_hahas|num_sads|num_angrys|Hour|  comments_predict|
+-------------+------------+----------+---------+---------+--------+---------+--------+----------+----+------------------+
|          386|        1912|       394|      246|      137|       1|        2|       0|         0|   1|1509.2665688753127|
|          255|         505|       187|      168|       80|       5|        1|       0|         1|   7| 300.7803058385849|
|         1730|         161|         4|     1710|       12|       8|        0|       0|         0|  11| 1389.285314297676|
|          237|          67|         1|      233|        3|       1|        0|       0|         0|   6|148.81175470352173|
|           43|          17|         0|       41|        1|       1|        0|       0|         0|   0|24.431980180740357|
|           10| 

In [ ]:
predict_values3 = df_predict3.select('comments_predict').rdd.flatMap(lambda x: x).collect()

In [ ]:
print(predict_values3)

[1509.2665688753127, 300.7803058385849, 1389.285314297676, 148.81175470352173, 24.431980180740357, 9.48111081123352, 107.59197225570679, 102.37436702251435, 1408.585259461403, 688.7022478818893, 38.79708640575409, 30.015625619888304, 878.7959464073181, 40.515461111068724, 677.506664466858, 531.7311802387237, 2254.926922893524, 35.03466074466705, 1097.5103040456772, 22.857742309570312, 67.60812501907348, 34.774064254760745, 118.12562146186829, 0.7437291145324707, 12.869198226928711, 1382.941720032692, 2471.485867333412, 19.396091556549074, 24.310610222816468, 940.0532574653625, 3.255836749076843, 74.98206470012664, 60.855890417099, 562.9539205789566, 161.43138148784638, 235.8293049097061, 2320.7710514307023, 728.8952317714691, 2.488475704193115, 101.87213251590728, 108.17771608829499, 400.9197550535202, 186.67167899608611, 18.045014667510987, 25.439884066581726, 11.021690773963929, 269.6227519989014, 21.94792683124542, 27.61054131984711, 48.49320056438446, 1194.1986968517303, 29.4925491

In [ ]:
df_predict3.select('num_comments','comments_predict').show()

+------------+------------------+
|num_comments|  comments_predict|
+------------+------------------+
|        1912|1509.2665688753127|
|         505| 300.7803058385849|
|         161| 1389.285314297676|
|          67|148.81175470352173|
|          17|24.431980180740357|
|           0|  9.48111081123352|
|          53|107.59197225570679|
|          20|102.37436702251435|
|        1814| 1408.585259461403|
|          90| 688.7022478818893|
|          66| 38.79708640575409|
|           2|30.015625619888304|
|        1051| 878.7959464073181|
|           0|40.515461111068724|
|         785|  677.506664466858|
|         660| 531.7311802387237|
|        2609| 2254.926922893524|
|           2| 35.03466074466705|
|        1315|1097.5103040456772|
|         239|22.857742309570312|
+------------+------------------+
only showing top 20 rows



In [ ]:
print("R_square: "+ str(R_square(df_predict3,'num_comments','comments_predict')))
print("RMSD: "+str(RMSD(df_predict3,data_test3.count(),'num_comments','comments_predict')))

R_square: 0.9452075159173892
RMSD: 362.4827201445099


In [ ]:
print(avg_error_rate2(df_predict3,'num_comments','comments_predict'))

12.810861787098414
